In [ ]:
%load_ext autoreload
%autoreload 2

from hydroml.training.train import train
from hydroml.config.config import Config, load_config
import hydroml.training.finetune as ft
from hydroml.models import get_model_from_path
from hydroml.data.camels_aus_ds import get_dataset
from hydroml.prediction.prediction import process_and_convert_dataloader_to_xarray
from hydroml.models.get_model_from_path import get_model_from_path
from hydroml.evaluation.metrics import get_metrics
# read basins.txt


with open('P://work//sho108//hydroml//workflows//basins.txt', 'r') as f:
    basins = f.read().splitlines()


config = Config(cal={'periods' : [['1985-01-01', '2014-01-01']], 'catchment_ids':basins[:4]}, #, 'weights': [1, 1, 1, 1, 1, 1, 1, 1, 1]
                val={'periods' : [['1985-01-01', '2014-01-01']], 'catchment_ids':basins[:4]}, #,
                max_epochs=150,
                lstm_hidden_size=64,
                batch_size=128,
                )

current_path, version = train(config)

dataset = get_dataset(config, 'val', is_train=True)
dataloader = dataset.to_dataloader()
model = get_model_from_path(current_path/version)

ds = process_and_convert_dataloader_to_xarray(dataloader, model, config)
m_after_reload_from_path = get_metrics(ds)



In [ ]:
from pathlib import Path
current_path = Path('P://work//sho108//hydroml//results//default//')
version = '241210095342_a18a'
m_after_updating_the_config = {}
m_after_finetuning = {}
config = load_config(current_path/version/'config.yaml')
model = get_model_from_path(current_path/version)
#model.freeze_all_layers()
#model.unfreeze_layers(['head'])
#model.unfreeze_all_layers()

model.lr = 1e-3



ds = process_and_convert_dataloader_to_xarray(dataloader, model, config)
before_finetuning = get_metrics(ds)

for catchment_id in basins[1:2]:
    #config = ft.update_config_for_per_catchment_finetune(config, cal_catchment_ids=[catchment_id], val_catchment_ids=[catchment_id], max_epochs=150)
    #load_model_from_ath


    ds = process_and_convert_dataloader_to_xarray(dataloader, model, config)
    m_after_updating_the_config[catchment_id] = get_metrics(ds)

    current_path, version = ft.finetune(model, config, finetune_directory='finetune', finetune_name=catchment_id)

    ds = process_and_convert_dataloader_to_xarray(dataloader, model, config)
    m_after_finetuning[catchment_id] = get_metrics(ds)

In [ ]:
#get forget bias
from matplotlib import pyplot as plt
#model = get_model_from_path(current_path/version)
plt.plot(model.get_forget_bias())



In [ ]:
catchment_id

In [ ]:

print('-- after reload from path')
print(m_after_reload_from_path.to_dataframe().loc[basins[1], 'nse'])
print('='*100)

print('-- before finetuning')
print(before_finetuning.to_dataframe().loc[basins[1], 'nse'])
print('='*100)

for catchment_id in [basins[1]]:
    #print('-- after updating the config')
    #print(f'Catchment ID: {catchment_id}')
    
    #print(m_after_updating_the_config[catchment_id].to_dataframe()['nse'])
    
    print('after finetuning')
    print(m_after_finetuning[catchment_id].to_dataframe().loc[catchment_id, 'nse'])
    print('-'*100)

